In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import json
import pandas as pd
import logging
from ipywidgets import Dropdown

<IPython.core.display.Javascript object>

In [3]:
import boto3
import sagemaker

<IPython.core.display.Javascript object>

In [4]:
boto3.set_stream_logger(name="botocore.credentials", level=logging.WARNING)

<IPython.core.display.Javascript object>

In [5]:
sess = sagemaker.Session()
region = sess.boto_region_name
print(region)

ap-southeast-2


<IPython.core.display.Javascript object>

In [6]:
# download JumpStart model_manifest file.
boto3.client("s3").download_file(
    f"jumpstart-cache-prod-{region}", "models_manifest.json", "models_manifest.json"
)
with open("models_manifest.json", "rb") as json_file:
    model_list = json.load(json_file)

print("number of models: ", len(model_list))

number of models:  2751


<IPython.core.display.Javascript object>

In [7]:
model_df = pd.DataFrame(model_list)
model_df.sample(20)

,model_id,version,min_version,spec_key
1723,tensorflow-ic-imagenet-mobilenet-v1-100-224-classification-4,3.0.1,2.80.0,community_models/tensorflow-ic-imagenet-mobilenet-v1-100-224-classification-4/specs_v3.0.1.json
217,huggingface-summarization-distilbart-cnn-6-6,1.0.0,2.75.0,community_models/huggingface-summarization-distilbart-cnn-6-6/specs_v1.0.0.json
2504,tensorflow-spc-electra-base-1,1.0.0,2.75.0,community_models/tensorflow-spc-electra-base-1/specs_v1.0.0.json
1813,tensorflow-ic-imagenet-mobilenet-v2-100-224-classification-4,3.0.1,2.80.0,community_models/tensorflow-ic-imagenet-mobilenet-v2-100-224-classification-4/specs_v3.0.1.json
407,huggingface-txt2img-hassanblend-hassanblend1-4,1.0.0,2.81.0,community_models/huggingface-txt2img-hassanblend-hassanblend1-4/specs_v1.0.0.json
...,...,...,...,...
567,mxnet-od-faster-rcnn-fpn-resnet50-v1b-coco,1.0.0,2.75.0,community_models/mxnet-od-faster-rcnn-fpn-resnet50-v1b-coco/specs_v1.0.0.json
90,huggingface-spc-bert-base-multilingual-cased,1.1.0,2.75.0,community_models/huggingface-spc-bert-base-multilingual-cased/specs_v1.1.0.json
491,lightgbm-classification-model,1.3.1,2.75.0,community_models/lightgbm-classification-model/specs_v1.3.1.json
1520,tensorflow-ic-imagenet-inception-v2-classification-4,3.0.0,2.80.0,community_models/tensorflow-ic-imagenet-inception-v2-classification-4/specs_v3.0.0.json


<IPython.core.display.Javascript object>

In [8]:
# filter-out all the Object Detection models from the manifest list.
od_models = []
for model in model_list:
    model_id = model["model_id"]
    if ("-od-" in model_id or "-od1-" in model_id) and model_id not in od_models:
        od_models.append(model_id)

print(f"Number of od models available for inference: {len(od_models)}")

Number of od models available for inference: 70


<IPython.core.display.Javascript object>

In [9]:
# display the model-ids in a dropdown to select a model for inference.
infer_model_dropdown = Dropdown(
    options=od_models,
    value="pytorch-od-nvidia-ssd",
    description="Select a model:",
    style={"description_width": "initial"},
    layout={"width": "max-content"},
)

<IPython.core.display.Javascript object>

In [10]:
display(infer_model_dropdown)


Dropdown(description='Select a model:', index=17, layout=Layout(width='max-content'), options=('mxnet-od-faste…

<IPython.core.display.Javascript object>

In [11]:
print(infer_model_dropdown.value)

pytorch-od-nvidia-ssd


<IPython.core.display.Javascript object>

In [12]:
# filter-out all the Image Classification models from the manifest list.
ic_models = []
for model in model_list:
    model_id = model["model_id"]
    if ("-ic-" in model_id) and model_id not in ic_models:
        ic_models.append(model_id)

print(f"Number of ic models available for inference: {len(ic_models)}")

Number of ic models available for inference: 162


<IPython.core.display.Javascript object>

In [13]:
# display the model-ids in a dropdown to select a model for inference.
infer_model_dropdown = Dropdown(
    options=ic_models,
    value="pytorch-ic-alexnet",
    description="Select a model:",
    style={"description_width": "initial"},
    layout={"width": "max-content"},
)

<IPython.core.display.Javascript object>

In [14]:
display(infer_model_dropdown)

Dropdown(description='Select a model:', layout=Layout(width='max-content'), options=('pytorch-ic-alexnet', 'py…

<IPython.core.display.Javascript object>

In [15]:
print(infer_model_dropdown.value)

pytorch-ic-alexnet


<IPython.core.display.Javascript object>